# H2O adsorption and diffusion in MOF-74Ni

******************
### Updates
2022/05/16  - minor revisions mainly to accomodate calculator and optimizer setups
******************

In this case, we will use MOF-74 (aka CPO-27) to observe the adsorption and diffusion behavior of water molecules with PFP.
MOF-74 has various versions of metals such as Ni, Co, Zn, and Mg, but all of them have a one-dimensional nanoporous structure bridged by DOBDC, leaving the metal part bare. 
Water molecules and other small molecules can be adsorbed here and are expected to be used as adsorbents, separation membrane materials, and catalyst materials.

MOF contains various elements and is sparse structure in general. It is high cost to calculate with DFT, and need huge effort to derive parameters to calculate with classical MD potential.
PFP enables us to perform various calculations such as molecular dynamics and Monte Carlo calculations in a short time without these challenging characteristics of MOF. 

This calculation case is published in both PFP paper & Matlantis website<br/>
Note that the calculation result may differ due to using the different PFP version, compared to paper.

 - [Towards universal neural network potential for material discovery applicable to arbitrary combination of 45 elements | Nature Communications](https://www.nature.com/articles/s41467-022-30687-9)
 - [Adsorption and dynamics of H2O molecules in MOF-74Ni | MATLANTIS](https://matlantis.com/calculation/adsorption-and-dynamics-of-h2o-molecules-in-mof-74ni)

First, import necessary libraries.

In [1]:
# Generatl import statements

# PFP　
import pfp_api_client
print(f"pfp_api_client: {pfp_api_client.__version__}")
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator

estimator = Estimator(calc_mode='CRYSTAL_PLUS_D3',model_version='v2.0.0') #DFT-D3
calculator = ASECalculator(estimator)

#print("estimator : ",estimator)
print("calc_mode: ",estimator.calc_mode)
#print("calculator : ",calculator)

# ASE
from ase.io import read,write,cif,gaussian,vasp,PickleTrajectory,xyz
from ase import Atoms, Atom
from ase.constraints import FixAtoms 
from ase.build import bulk
from ase.visualize import view
from ase.build import surface
from ase.constraints import StrainFilter,ExpCellFilter,UnitCellFilter
from ase.optimize import LBFGS,FIRE,MDMin
from ase.io.trajectory import Trajectory
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution,Stationary
from ase.md import MDLogger
from ase import units

# Other external packages
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import itertools
import json
from IPython.display import Image, display_png
import nglview as nv
import ipywidgets as widgets

pfp_api_client: 1.3.1
calc_mode:  CRYSTAL_PLUS_D3


In [2]:
import os

indir = Path("input")
outdir = Path("output")
os.makedirs(outdir, exist_ok=True)

Optimize each structure: 

 - water molecule
 - MOF-74
 - adsorption structure of MOF-74 and water molecules

In [3]:
# h2o
h2o = read(indir/"H2O.POSCAR")
h2o.set_calculator(calculator)

fmax = 0.005 # Force convergence criterion in eV/Å
opt = LBFGS(h2o)
opt.run(fmax=fmax)
write(indir/"H2O_opt.POSCAR",h2o)
epot_h2o = h2o.get_potential_energy() 
epot_h2o

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 01:23:19      -10.027575*       0.3033
LBFGS:    1 01:23:20      -10.029338*       0.1671
LBFGS:    2 01:23:20      -10.031338*       0.1400
LBFGS:    3 01:23:20      -10.031676*       0.0411
LBFGS:    4 01:23:20      -10.031692*       0.0129
LBFGS:    5 01:23:20      -10.031693*       0.0006


-10.031692870771481

For MOF-74 and structures with water molecules adsorbed on MOF-74, we will try to optimize both cell size and atomic coordinates simultaneously using `ExpCellFilter` for optimization.

In [4]:
#MOF-74
# "Ni", "Co", "Zn", "Mg" can be used.
metal = "Ni"

mof = read(indir/f"MOF-74{metal}.POSCAR")
mof.set_calculator(calculator)

fmax = 0.005 # Force convergence criterion in eV/Å
ef = UnitCellFilter(mof)
opt = LBFGS(ef)
opt.run(fmax=fmax)
write(indir/f"MOF-74{metal}_opt.POSCAR",mof)
epot_mof = mof.get_potential_energy() 
epot_mof

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 01:23:20     -938.647937*       1.0633
LBFGS:    1 01:23:20     -939.093555*       1.0454
LBFGS:    2 01:23:20     -939.512015*       0.9515
LBFGS:    3 01:23:21     -940.521662*       1.1416
LBFGS:    4 01:23:21     -940.874754*       1.0059
LBFGS:    5 01:23:21     -941.182285*       0.6007
LBFGS:    6 01:23:21     -941.284304*       0.4654
LBFGS:    7 01:23:21     -941.375802*       0.3667
LBFGS:    8 01:23:21     -941.447964*       0.3279
LBFGS:    9 01:23:21     -941.500945*       0.2432
LBFGS:   10 01:23:21     -941.571729*       0.2713
LBFGS:   11 01:23:21     -941.618417*       0.2840
LBFGS:   12 01:23:21     -941.675024*       0.2440
LBFGS:   13 01:23:21     -941.728307*       0.2243
LBFGS:   14 01:23:21     -941.781822*       0.2137
LBFGS:   15 01:23:22     -941.827006*       0.2245
LBFGS:   16 01:23:22     -941.869748*       0.2273
LBFGS:   17 01:23:22     -941.9087

-943.5865025986745

Check that the structure after optimization is OK.

In [5]:
from ase.visualize import view

v = view(mof, viewer='ngl')
v.view.add_representation("ball+stick")
display(v)

Next is the structure of water molecules adsorbed on MOF-74.

In [6]:
# MOF-74 with H2O adsorbed
mof_and_h2o = read(indir/f"MOF-74{metal}_and_18H2O.POSCAR")
mof_and_h2o.set_calculator(calculator)

fmax = 0.005 # Force convergence criterion in eV/Å
ef = UnitCellFilter(mof_and_h2o)
opt = LBFGS(ef)
opt.run(fmax=fmax)

write(outdir/f"MOF-74{metal}_and_18H2O_opt.POSCAR", mof_and_h2o)
epot_mof_and_h2o = mof_and_h2o.get_potential_energy() 
epot_mof_and_h2o

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 01:23:43    -1133.914280*       0.5052
LBFGS:    1 01:23:43    -1134.013488*       0.4085
LBFGS:    2 01:23:43    -1134.077315*       0.4079
LBFGS:    3 01:23:43    -1134.289833*       0.6907
LBFGS:    4 01:23:43    -1134.387515*       0.5115
LBFGS:    5 01:23:43    -1134.472872*       0.4921
LBFGS:    6 01:23:43    -1134.513683*       0.4861
LBFGS:    7 01:23:44    -1134.559278*       0.3792
LBFGS:    8 01:23:44    -1134.606010*       0.3445
LBFGS:    9 01:23:44    -1134.634616*       0.3588
LBFGS:   10 01:23:44    -1134.684967*       0.4362
LBFGS:   11 01:23:44    -1134.656371*       0.9924
LBFGS:   12 01:23:44    -1134.781539*       0.4804
LBFGS:   13 01:23:44    -1134.835227*       0.5047
LBFGS:   14 01:23:44    -1134.874022*       0.2864
LBFGS:   15 01:23:45    -1134.946591*       0.2880
LBFGS:   16 01:23:45    -1134.998774*       0.3903
LBFGS:   17 01:23:45    -1135.0508

-1138.3690003749307

In [7]:
from ase.visualize import view

view(mof_and_h2o,viewer="ngl")
v.view.add_representation("ball+stick")
display(v)

From the energy values obtained here, we can determine the adsorption Energy. 
Since one water molecule is adsorbed on each Ni atom, there are 18 water molecules in this structure.

In [8]:
# Updated data:
(epot_mof_and_h2o - epot_mof - 18*epot_h2o)/18.0

-0.789557005687194

In this PFP version, a value of -0.78 eV per water molecule was obtained (-0.67eV in the old version).

In [9]:
# Updated data:

lat1 = mof.cell.cellpar()
lat2 = mof_and_h2o.cell.cellpar()
v1 = mof.get_volume()
v2 = mof_and_h2o.get_volume()
dv = (v2-v1)/v1*100

print("Lattice constant & angle：")
print("　　MOF only    :{0:.3f} {1:.3f} {2:.3f} {3:.3f} {4:.3f} {5:.3f}".format(*lat1))
print("　　MOF & 18H2O :{0:.3f} {1:.3f} {2:.3f} {3:.3f} {4:.3f} {5:.3f}".format(*lat2))
print("Volume:")
print("　　MOF      : {0:.2f}".format(v1))
print("　　MOF&18H2O: {0:.2f}".format(v2))
print("　　Diff     : {0:+.2f} %".format(dv))

Lattice constant & angle：
　　MOF only    :25.859 25.859 6.671 89.996 90.003 119.985
　　MOF & 18H2O :26.025 26.068 6.854 90.002 89.966 119.986
Volume:
　　MOF      : 3863.55
　　MOF&18H2O: 4027.26
　　Diff     : +4.24 %


Old version:

```
Lattice constant & angle：
　　MOF only    :25.682 25.678 6.796 89.998 90.005 119.982
　　MOF & 18H2O :26.071 26.116 6.952 90.006 90.008 120.021
Volume:
　　MOF      : 3882.23
　　MOF&18H2O: 4098.45
　　Diff     : +5.57 %
```

A comparison of before and after water molecule adsorption shows that the lattice constant increases by 1-2% and the volume by >4% when water molecules are adsorbed.

Next, we follow the behavior of adsorbed molecules with molecular dynamics simulations.

In [12]:
T = 573
num_md_steps  = 200000 
num_int_print = 200
time_step = 0.5 * units.fs

MaxwellBoltzmannDistribution(mof_and_h2o, T*units.kB)
Stationary(mof_and_h2o)

dyn = Langevin(mof_and_h2o, time_step, T * units.kB, 0.02, logfile=outdir/f"md_MOF-74{metal}_and_H2O_573K_nvt.log")

def printenergy(a=mof_and_h2o):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    #a.wrap()
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

dyn.attach(printenergy, interval=num_int_print)

# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory(outdir/f"md_MOF-74{metal}_and_H2O_573K_nvt.traj", 'w', mof_and_h2o)
dyn.attach(traj.write, interval=num_int_print)

# Now run the dynamics
printenergy()
dyn.run(num_md_steps)

/home/jovyan/.local/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Energy per atom: Epot = -5.270eV  Ekin = 0.075eV (T=581K)  Etot = -5.195eV
Energy per atom: Epot = -5.270eV  Ekin = 0.075eV (T=581K)  Etot = -5.195eV
Energy per atom: Epot = -5.225eV  Ekin = 0.043eV (T=333K)  Etot = -5.182eV
Energy per atom: Epot = -5.218eV  Ekin = 0.046eV (T=355K)  Etot = -5.173eV
Energy per atom: Epot = -5.215eV  Ekin = 0.051eV (T=391K)  Etot = -5.165eV
Energy per atom: Epot = -5.213eV  Ekin = 0.054eV (T=417K)  Etot = -5.159eV
Energy per atom: Epot = -5.211eV  Ekin = 0.058eV (T=448K)  Etot = -5.153eV
Energy per atom: Epot = -5.209eV  Ekin = 0.058eV (T=453K)  Etot = -5.150eV
Energy per atom: Epot = -5.204eV  Ekin = 0.065eV (T=507K)  Etot = -5.139eV
Energy per atom: Epot = -5.202eV  Ekin = 0.064eV (T=494K)  Etot = -5.138eV
Energy per atom: Epot = -5.199eV  Ekin = 0.065eV (T=503K)  Etot = -5.134eV
Energy per atom: Epot = -5.203eV  Ekin = 0.067eV (T=517K)  Etot = -5.136eV
Energy per atom: Epot = -5.204eV  Ekin = 0.071eV (T=553K)  Etot = -5.132eV
Energy per atom: Epot = -

True